In [2]:
%pylab inline
import pandas as pd

import numpy as np
from __future__ import division
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

import logging
logger = logging.getLogger()

Populating the interactive namespace from numpy and matplotlib


15 Dynamic Programming
============

for optimization problems:   

1. divide-and-conquer    
   partition into **disjoint** subproblems, solve recursively, and then combine.   
 
2. dynamic programming    
   subploems **overlap**    
   developing by four steps:    
   1. Characterize the **structure** of an optimal solution.   
   2. **Recursively** define the value of an optimal solution.    
   3. Compute the value of an optimal solution, typically in a **bottom-up** fashion.    
   4. [opt] Construct an optimal solution from computed information.
   


### 15.1 Rod cutting

##### rod-cutting problem
Given:   
1. a rod of length $n$ inches.   
2. a table of prices $p_i$ for $i = 1, 2, \cdots, n$  

Problem:   
determine the __maximum revenue__ $r_n$ obtainable by cutting up the rod and selling the pieces.

Solution:    
$$r_n = max_{1 \leq i \leq n} (p_i + r_{n-i})$$

Implementation:   
1. recursive top-down    
2. dynamic programming   
    + **optimal substructure**: optimal solutions to a problem incorporate optimal solutions to related subproblems.    
    + uses additional memory to save computation time     
    + two ways:
        1. top-down with memorization      
            + it “remembers” what results it has computed previously.   
        2. bottom-up method.    
            + We sort the subproblems by size and solve them in size order, smallest first.   
            + The bottom-up approach often has much better constant factors, since it has less overhead for procedure calls.
    + **subproblems graphs**, as shown in Fig (15.4).     
        - A dynamic-programming approach runs in **polynomial** time when **the number of distinct subproblems** involved is **polynomial** in the input size and we can solve each such subproblem in **polynomial** time.    
        - namely, the running time of dynamic programming is **linear** in the number of vertices and edges.
    + **Reconstructing a solution**:    
      We can extend the dynamic-programming approach to record not only the optimal value computed for each subproblem, but also a choice that led to the optimal value. With this information, we can readily print an optimal solution.
￼

In [3]:
price = np.array([1, 5, 8, 9, 10, 17, 17, 20, 24, 30])

print('length i: {}'.format(range(len(price))))
print('price p_i: {}'.format(price))

def bottom_up_cut_rod(p_, n):
    """Solve cut_rod problem by bottom-up dynamic programming.
    
    """
    assert (len(p_) >= n), 'n should be less than the max length {}'.format(len(p))
    
    p = np.insert(p_, 0, 0) #set price_0 = 0 
    
    r = np.zeros(n+1)
    
    for k in range(1, n+1):
        q = -np.inf
        for m in range(1, k+1):
            q = max(q, p[m]+r[k-m])
        r[k] = q
        
    return r[-1]


print('max price: {}'.format([bottom_up_cut_rod(price, n) for n in range(1, len(price)+1)]))

length i: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
price p_i: [ 1  5  8  9 10 17 17 20 24 30]
max price: [1.0, 5.0, 8.0, 10.0, 13.0, 17.0, 18.0, 22.0, 25.0, 30.0]


In [21]:
logger.setLevel('DEBUG')

arr = np.array([-2, 8, 4, -5, 9, 3, -1, 4, 5, -17, 1, -9, 6])

def max_sum_sequence(x, display=False):
    a = np.zeros(len(x)+1)
    a[0] = -np.inf
    e = a.copy()
    
    x = np.insert(x, 0, 0)
    
    for k in range(1, len(x)):
        e[k] = max(x[k], x[k]+e[k-1])
        a[k] = max(x[k], x[k]+e[k-1], a[k-1])
        
    logging.debug('\n e: {} \n a:{} \n'.format(e, a))
    
    seq = []
    seq_end = np.nonzero(a==a[-1])[0][0]
    logging.debug('seq_end ins: {}, value: {}\n'.format(seq_end, ))


        
    return a[-1]

max_sum_sequence(arr)

DEBUG:root:
 e: [-inf  -2.   8.  12.   7.  16.  19.  18.  22.  27.  10.  11.   2.   8.] 
 a:[-inf  -2.   8.  12.  12.  16.  19.  19.  22.  27.  27.  27.  27.  27.] 

DEBUG:root:seq_end: 9



27.0

### 15.2 Martrix-chain multiplication

##### matrix-chain multiplication problem
**given** a chain $<A_1, A_2, \cdots, A_n>$ of $n$ matrices, where for $i = 1, 2, \cdots, n$, matrix $A_i$ has dimension $p_{i-1} \times p_i$,     
**problem**: fully parenthesize the product $A_1 A_2 \cdots A_n$ in a way that minimizes the number of scalar multiplicatons.

**solutions**: 

1. exhaustively checking all possible parenthesizations. $\Omega(2^n)$      
    \begin{equation}
        P(n) = \begin{cases}
            1 & \quad \text{if } n = 1 \\
            \sum_{k=1}^{n-1} P(k) P(n-k) & \quad \text{if } n \geq 2 \\
        \end{cases}
    \end{equation}

2. Applying dynamic programming. $\Omega (n^3)$    
    define $A_{i \dotso j} = A_i A_{i+1} \dotsm A_j$    
    
    1. The structure of an optimal parenthesization.    
       suppose $A_{i \dotso k} A_{k \dotso j}$ is the optimal solution of $<A_i, \dotsm, A_j>$, where $i \leq k < j$.   
       then $A_{i \dotso k}$ must be the optimal solution of $<A_i, \dotsm, A_k>$, as well as $A_{k \dots j}$.
       
       Proof:     
       if existing $Cost(\hat{A}_{i \dotso k}) < Cost(A_{i \dotso k})$,     
       then $\hat{A}_{i \dotso k} A_{k \dotso j}$ should be the optimal solution ==> contradiction.    
       
    2. A recursive solution.    
       Let $m[i,j]$ be the minimum number of scalar multiplications needed to compute the matrix $A_{i \dotso j}$.  
       
       \begin{equation}
           m[i,j] = \begin{cases}
               0 & \quad \text{if } i = j \\
               min_{i \leq k < j} {m[i,k] + m[k+1,j] + p_{i-1} p_k p_j} & \quad \text{if } i < j
           \end{cases}
       \end{equation} 
       
    3. Computing the optimal costs.   
       $m[i,j]$ depends on all $m[i,k], m[k+1,j]$.    
       hence, the algorithm should fill in the talbe $m$ in a manner that corresponds to solving the parenthesizaiton problem on matrix chains of incresing lenghth.    
       
    4. Constructing an optimal solution.    
       construct table $s$ whose rows is $1, \dotsm, n-1$ and columns is $2, \dotsm, n$.    
       Each entry $s[i,j]$ records the optimal solution $k$ for $A_{i \dotso j}$.